<a href="https://colab.research.google.com/github/Hisernberg/ai-agent-projects/blob/main/Copy_of_Module4Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > output.log 2>&1 &
!ollama pull phi4

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manife

In [1]:
!pip install ollama faiss-cpu sentence-transformers numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

Unoptimized Agent

In [3]:
import ollama
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import time
import logging
from typing import List, Tuple

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class SimpleAgent:
    def __init__(self):
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.dimension = 384  # Dimension of the embedding model
        self.index = faiss.IndexFlatL2(self.dimension)
        self.documents = []
        self.token_usage = 0
        self.query_times = []

    def embed_text(self, text: str) -> np.ndarray:
        """Generate embedding for a given text."""
        return self.embedding_model.encode([text])[0]

    def add_document(self, document: str):
        """Add a document to the FAISS index."""
        start_time = time.time()
        embedding = self.embed_text(document)
        self.index.add(np.array([embedding], dtype=np.float32))
        self.documents.append(document)
        logger.info(f"Added document. Time: {time.time() - start_time:.4f}s")

    def query(self, question: str, k: int = 1) -> Tuple[str, int]:
        """Query the agent with a question."""
        start_time = time.time()

        # Generate embedding for the question
        question_embedding = self.embed_text(question)

        # Search FAISS index
        distances, indices = self.index.search(np.array([question_embedding], dtype=np.float32), k)

        # Retrieve relevant documents
        context = [self.documents[idx] for idx in indices[0] if idx < len(self.documents)]
        context_text = "\n".join(context)

        # Prepare prompt (unoptimized: verbose and redundant)
        prompt = f"""
        You are a helpful assistant. Given the following context and question, provide a detailed answer.

        Context:
        {context_text}

        Question:
        {question}

        Please provide a comprehensive response with all relevant details.
        """

        # Call Ollama Phi-4
        response = ollama.chat(
            model='phi4',
            messages=[{'role': 'user', 'content': prompt}]
        )

        # Track token usage (approximate)
        prompt_tokens = len(prompt.split())
        response_tokens = len(response['message']['content'].split())
        total_tokens = prompt_tokens + response_tokens
        self.token_usage += total_tokens

        # Track query time
        query_time = time.time() - start_time
        self.query_times.append(query_time)

        logger.info(f"Query processed. Time: {query_time:.4f}s, Tokens: {total_tokens}")

        return response['message']['content'], total_tokens

    def get_performance_metrics(self) -> dict:
        """Return performance metrics."""
        return {
            'total_token_usage': self.token_usage,
            'average_query_time': np.mean(self.query_times) if self.query_times else 0,
            'number_of_queries': len(self.query_times)
        }

# Example usage
if __name__ == "__main__":
    agent = SimpleAgent()

    # Add sample documents
    documents = [
        "The capital of France is Paris.",
        "Python is a popular programming language.",
        "The sun is a star."
    ]

    for doc in documents:
        agent.add_document(doc)

    # Test queries
    queries = [
        "What is the capital of France?",
        "What is Python?",
        "Is the sun a star?"
    ]

    for query in queries:
        answer, tokens = agent.query(query)
        print(f"Query: {query}")
        print(f"Answer: {answer}")
        print(f"Tokens used: {tokens}\n")

    # Print performance metrics
    metrics = agent.get_performance_metrics()
    print("Performance Metrics:")
    print(f"Total Token Usage: {metrics['total_token_usage']}")
    print(f"Average Query Time: {metrics['average_query_time']:.4f}s")
    print(f"Number of Queries: {metrics['number_of_queries']}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Query: What is the capital of France?
Answer: The capital of France is Paris. Known as "La Ville Lumière" or "The City of Light," Paris holds significant historical, cultural, and political importance not only in France but also globally.

**Geographical Location:**
Paris is situated in the north-central part of France along the Seine River. It serves as a major European city and an important hub for finance, commerce, fashion, science, and the arts.

**Historical Significance:**
Paris has a rich history that dates back to its founding by the Parisii tribe around 250 BC. Over the centuries, it became the heart of France’s cultural renaissance during the Age of Enlightenment in the 18th century. The city is home to numerous historical landmarks such as Notre-Dame Cathedral, the Eiffel Tower (constructed for the 1889 World's Fair), and the Louvre Museum.

**Cultural Impact:**
Paris is often considered a global center for art, fashion, gastronomy, and culture. It hosts major international

You Answer: Optimized Agent with performance metrics

Optimization Techniques Implemented:
(List down the techniques you've used)

*   


optimized agent


In [4]:
#You need to write the agent here in this cell with optimization
import ollama
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import time
import logging
from typing import List, Tuple

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class SimpleAgent:
    def __init__(self):
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.dimension = 384  # Dimension of the embedding model
        self.index = faiss.IndexFlatL2(self.dimension)
        self.documents = []
        self.token_usage = 0
        self.query_times = []

    def embed_text(self, text: str) -> np.ndarray:
        """Generate embedding for a given text."""
        return self.embedding_model.encode([text])[0]

    def add_document(self, document: str):
        """Add a single document to the FAISS index."""
        self.add_documents([document])

    def add_documents(self, documents: List[str]):
        """Add multiple documents to the FAISS index."""
        start_time = time.time()
        embeddings = self.embedding_model.encode(documents)
        self.index.add(np.array(embeddings, dtype=np.float32))
        self.documents.extend(documents)
        logger.info(f"Added {len(documents)} documents. Time: {time.time() - start_time:.4f}s")

    def query(self, question: str, k: int = 1) -> Tuple[str, int]:
        """Query the agent with a question."""
        start_time = time.time()

        # Generate embedding for the question
        question_embedding = self.embed_text(question)

        # Search FAISS index
        distances, indices = self.index.search(np.array([question_embedding], dtype=np.float32), k)

        # Retrieve relevant documents
        context = [self.documents[idx] for idx in indices[0] if idx < len(self.documents)]
        context_text = "\n".join(context)

        # Optimized shorter prompt
        prompt = f"""
Answer the question using the context below.

Context:
{context_text}

Question:
{question}
"""

        try:
            # Call Ollama Phi-4
            response = ollama.chat(
                model='phi4',
                messages=[{'role': 'user', 'content': prompt}]
            )
        except Exception as e:
            logger.error(f"Ollama call failed: {e}")
            return "I'm sorry, I couldn't process your request.", 0

        # Track token usage (approximate)
        prompt_tokens = len(prompt.split())
        response_tokens = len(response['message']['content'].split())
        total_tokens = prompt_tokens + response_tokens
        self.token_usage += total_tokens

        # Track query time
        query_time = time.time() - start_time
        self.query_times.append(query_time)

        logger.info(f"Query processed. Time: {query_time:.4f}s, Tokens: {total_tokens}")

        return response['message']['content'], total_tokens

    def get_performance_metrics(self) -> dict:
        """Return performance metrics."""
        return {
            'total_token_usage': self.token_usage,
            'average_query_time': np.mean(self.query_times) if self.query_times else 0,
            'number_of_queries': len(self.query_times)
        }

# Example usage
if __name__ == "__main__":
    agent = SimpleAgent()

    # Add sample documents
    documents = [
        "The capital of France is Paris.",
        "Python is a popular programming language.",
        "The sun is a star."
    ]
    agent.add_documents(documents)

    # Test queries
    queries = [
        "What is the capital of France?",
        "What is Python?",
        "Is the sun a star?"
    ]

    for query in queries:
        answer, tokens = agent.query(query, k=1)
        print(f"Query: {query}")
        print(f"Answer: {answer}")
        print(f"Tokens used: {tokens}\n")

    # Print performance metrics
    metrics = agent.get_performance_metrics()
    print("Performance Metrics:")
    print(f"Total Token Usage: {metrics['total_token_usage']}")
    print(f"Average Query Time: {metrics['average_query_time']:.4f}s")
    print(f"Number of Queries: {metrics['number_of_queries']}")


Query: What is the capital of France?
Answer: The capital of France is Paris.
Tokens used: 27

Query: What is Python?
Answer: Python is a high-level, interpreted programming language known for its simplicity and readability. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python has a comprehensive standard library that supports many common programming tasks, making it versatile and easy to use for various applications such as web development, data analysis, artificial intelligence, scientific computing, and more. Its popularity is also due to its large and active community, which contributes to numerous open-source projects and libraries.
Tokens used: 95

Query: Is the sun a star?
Answer: Yes, according to the provided context, the sun is a star.
Tokens used: 30

Performance Metrics:
Total Token Usage: 152
Average Query Time: 3.2073s
Number of Queries: 3


Performance Improvement: (Comparision)